In [1]:

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
import scipy.io as sio
#import matplotlib.pyplot as plt
#added
import h5py

import os

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:

sess=tf.Session()
saver = tf.train.import_meta_graph('./CNN3d-tflayers-model/model.ckpt-10.meta')

saver.restore(sess, './CNN3d-tflayers-model/model.ckpt-10')

INFO:tensorflow:Restoring parameters from ./CNN3d-tflayers-model/model.ckpt-10


In [3]:
#get the default graph
graph = tf.get_default_graph()

In [4]:
for op in graph.get_operations():
    print(op.name)

tf_x
tf_y
is_train
input_x_3d_volumes/shape
input_x_3d_volumes
input_y_onehot/on_value
input_y_onehot/off_value
input_y_onehot/depth
input_y_onehot
conv3d/kernel/Initializer/random_uniform/shape
conv3d/kernel/Initializer/random_uniform/min
conv3d/kernel/Initializer/random_uniform/max
conv3d/kernel/Initializer/random_uniform/RandomUniform
conv3d/kernel/Initializer/random_uniform/sub
conv3d/kernel/Initializer/random_uniform/mul
conv3d/kernel/Initializer/random_uniform
conv3d/kernel
conv3d/kernel/Assign
conv3d/kernel/read
conv3d/bias/Initializer/zeros
conv3d/bias
conv3d/bias/Assign
conv3d/bias/read
conv3d/Conv3D
conv3d/BiasAdd
conv3d/Relu
max_pooling3d/MaxPool3D
conv3d_1/kernel/Initializer/random_uniform/shape
conv3d_1/kernel/Initializer/random_uniform/min
conv3d_1/kernel/Initializer/random_uniform/max
conv3d_1/kernel/Initializer/random_uniform/RandomUniform
conv3d_1/kernel/Initializer/random_uniform/sub
conv3d_1/kernel/Initializer/random_uniform/mul
conv3d_1/kernel/Initializer/random_uni

In [5]:
vols = h5py.File("/scratch/hterborg/brute_force/Thesis/3_runs_15_train_1_test_dataset.mat",'r')
                 

In [6]:

X_train, y_train, X_test, y_test = vols['train_x'], vols['train_y'], vols['test_x'], vols['test_y']

In [7]:
print(np.shape(y_train),np.shape(y_test))

(1, 4645) (1, 317)


In [8]:
X_train=np.transpose(X_train)
y_train=np.transpose(y_train)
X_test=np.transpose(X_test)
y_test=np.transpose(y_test)

In [9]:

x_mean = np.mean(X_train)
x_std = np.std(X_train)
X_tr_centered = (X_train - x_mean)/x_std
X_ts_centered = (X_test - x_mean)/x_std

In [10]:
X_tr_centered.shape,  X_ts_centered.shape

((4645, 79, 95, 79), (317, 79, 95, 79))

In [11]:

y_train = y_train.flatten()
y_test = y_test.flatten()
y_train.shape,  y_test.shape

((4645,), (317,))

In [12]:
tf_x = graph.get_tensor_by_name("tf_x:0") # shape:(none,53,63,46)
is_train = graph.get_tensor_by_name("is_train:0")

In [13]:
output_tensor = graph.get_tensor_by_name('dense_1/Sigmoid:0')

In [14]:
 #tf.gradients(output_tensor[0][output_index],tf_x)[0]

In [15]:
#print(y_test[123])

In [16]:
#for all i = 1 than mean
# i = 123
# input_image = X_ts_centered[i,:]
# output_index = int(y_test[i])

# # define the calculation
# cal = tf.gradients(output_tensor[0][output_index],tf_x)[0]

# # expand dimension of the input image fit to the placeholder 'tf_x'
# x_value = np.expand_dims(input_image, axis=0)

# # get the gradients
# gradients = sess.run(cal, {tf_x:x_value,is_train:False})

# # to absolute value
# tp = gradients.squeeze()
# tp = np.absolute(tp)

In [17]:
#y_test[26]

In [18]:
bla= [i for i, e in enumerate(y_test) if e == 0]
print(np.shape(bla),np.shape(y_test))

(102,) (317,)


In [19]:
onecondition = [i for i, e in enumerate(y_test) if e ==0]

In [20]:
np.shape(onecondition)

(102,)

In [21]:
#del tp,alltp, gradients,x_value,cal, output_index,input_image

In [22]:
#del i

In [23]:
alltp = np.zeros([len(onecondition),79,95,79])

In [24]:

for i in range(len(onecondition)):

    input_image = X_ts_centered[onecondition[i],:]
    output_index = int(y_test[onecondition[i]])

    # define the calculation
    cal = tf.gradients(output_tensor[0][output_index],tf_x)[0]

    # expand dimension of the input image fit to the placeholder 'tf_x'
    x_value = np.expand_dims(input_image, axis=0)

    # get the gradients
    gradients = sess.run(cal, {tf_x:x_value,is_train:False})

    # to absolute value
    tp = gradients.squeeze()
    tp = np.absolute(tp)
    alltp[i]=tp


In [25]:
i

101

In [26]:
print(np.shape(alltp))

(102, 79, 95, 79)


In [27]:
meantp = np.mean(alltp,axis=0)

In [28]:
print(np.shape(meantp))

(79, 95, 79)


In [29]:
np.save('scrambled_OSD_10_mean_tp',meantp)

In [30]:
np.save('scrambled_OSD_10_alltp',alltp)